In [1]:
import argparse

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )  # cuda:number or cpu
        self.data = "gdsc2"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    0,  # Cell
    # 1  # Drug
]

load gdsc2


In [5]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [6]:
from joblib import Parallel, delayed
from tqdm import tqdm

n_kfold = 1
n_jobs = 50  # 並列数


def process_iteration(dim, target_index, seed, args):
    """各反復処理をカプセル化した関数"""
    if dim:
        if drug_sum[target_index] < 10:
            return None, None
    else:
        if cell_sum[target_index] < 10:
            return None, None

    fold_results = []
    for fold in range(n_kfold):
        true_data, predict_data = nihgcn_new(
            cell_exprs=exprs,
            drug_finger=drug_finger,
            res_mat=res,
            null_mask=null_mask,
            target_dim=dim,
            target_index=target_index,
            evaluate_fun=roc_auc,
            args=args,
            seed=seed,
        )
        fold_results.append((true_data, predict_data))

    return fold_results


# 並列処理の実行
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()

for dim in target_dim:
    # 全タスクを事前に生成
    tasks = [
        (dim, target_index, seed, args)
        for seed, target_index in enumerate(np.arange(res.shape[dim]))
    ]

    # 並列実行（プログレスバー付き）
    results = Parallel(n_jobs=n_jobs, verbose=0, prefer="threads")(
        delayed(process_iteration)(*task)
        for task in tqdm(tasks, desc=f"Processing dim {dim}")
    )

    # 結果の結合
    for fold_results in results:
        if fold_results is None:
            continue
        for true_data, predict_data in fold_results:
            true_data_s = pd.concat(
                [true_data_s, translate_result(true_data)],
                ignore_index=True,
                copy=False,  # メモリ節約のため
            )
            predict_data_s = pd.concat(
                [predict_data_s, translate_result(predict_data)],
                ignore_index=True,
                copy=False,
            )

Processing dim 0:   0%|          | 0/910 [00:00<?, ?it/s]

epoch:   0 loss:0.704774 auc:0.4911
epoch:   0 loss:0.702121 auc:0.5513
epoch:   0 loss:0.704717 auc:0.6457
epoch:   0 loss:0.707030 auc:0.4475
epoch:   0 loss:0.710614 auc:0.3345
epoch:   0 loss:0.698764 auc:0.4452
epoch:   0 loss:0.702891 auc:0.3217
epoch:   0 loss:0.695418 auc:0.5845
epoch:   0 loss:0.690566 auc:0.4977
epoch:   0 loss:0.706130 auc:0.5718
epoch:   0 loss:0.701533 auc:0.5511
epoch:   0 loss:0.694574 auc:0.5224
epoch:   0 loss:0.704751 auc:0.4405
epoch:   0 loss:0.709357 auc:0.3952
epoch:   0 loss:0.705153 auc:0.6278
epoch:   0 loss:0.696670 auc:0.3563
epoch:   0 loss:0.696496 auc:0.3984
epoch:   0 loss:0.701929 auc:0.4916
epoch:   0 loss:0.687916 auc:0.6497
epoch:   0 loss:0.695700 auc:0.4949
epoch:   0 loss:0.692385 auc:0.4559
epoch:   0 loss:0.701390 auc:0.6195
epoch:   0 loss:0.711387 auc:0.6597
epoch:   0 loss:0.707878 auc:0.5918
epoch:   0 loss:0.695575 auc:0.4716
epoch:   0 loss:0.705569 auc:0.4757
epoch:   0 loss:0.693635 auc:0.7031
epoch:   0 loss:0.694222 auc

epoch: 280 loss:0.078694 auc:0.9896
epoch: 200 loss:0.086570 auc:0.9799
epoch: 220 loss:0.083498 auc:0.9629
epoch: 120 loss:0.102425 auc:0.9813
epoch: 160 loss:0.093737 auc:0.9911
epoch: 200 loss:0.086702 auc:0.9964
epoch: 220 loss:0.083626 auc:0.9876
epoch: 220 loss:0.085104 auc:0.9905
epoch: 240 loss:0.085299 auc:0.9719
epoch: 140epoch: 140 loss:0.097802 auc:0.9394
 loss:0.097320 auc:0.9792
epoch: 260 loss:0.080682 auc:0.9412
epoch: 280 loss:0.078408 auc:0.9576
epoch: 300 loss:0.078179 auc:0.9896
epoch: 240 loss:0.081820 auc:0.9632
epoch: 200 loss:0.087935 auc:0.9972
epoch: 220 loss:0.084290 auc:0.9792


KeyboardInterrupt: 

epoch: 220 loss:0.083585 auc:0.9964
epoch: 240 loss:0.081239 auc:0.9901
epoch: 260 loss:0.080234 auc:0.9731
epoch: 280 loss:0.079066 auc:0.9448
epoch: 180 loss:0.089340 auc:0.9916
epoch: 300 loss:0.078317 auc:0.9580
epoch: 320 loss:0.077068 auc:0.9896
epoch: 240 loss:0.081684 auc:0.9898
epoch:  60 loss:0.122851 auc:0.9764
epoch: 260 loss:0.080931 auc:0.9615
epoch: 240 loss:0.081558 auc:0.9785
epoch: 160 loss:0.092621 auc:0.9792
epoch:  80 loss:0.113934 auc:0.9905
epoch:  60 loss:0.124052 auc:0.9648
epoch:  60 loss:0.124936 auc:0.9984
epoch: 220 loss:0.083971 auc:0.9972


In [ ]:
true_data_s.to_csv(f"new_cell_true_{args.data}.csv")
predict_data_s.to_csv(f"new_cell_pred_{args.data}.csv")